# Introduction to Text Formats

-----

We have already seen basic text formats when first learning to read and write data to a file. But one of the most important tasks when starting a data analysis project is understanding the format of a data file and how to best extract the necessary information from the data, whatever the format. Furthermore, problems can often arise when performing this task, thus a solid understanding of these concepts is necessary in order to minimize subsequent challenges. In this notebook, we explore different text data format, and present how to read and write data in this format by using standard, built-in Python tools.

-----




## Table of Contents

[Data Acquisition](#Data-Acquisition)

[Fixed-Width Format Text](#Fixed-Width-Format-Text)

[Delimiter Separated Values](#Delimiter-Separated-Values)


-----
[[Back to TOC]](#Table-of-Contents)

## Data Acquisition

Before we begin, however, we need to acquire test data to be able to have something that we can write and read to different text based formats. Note, the data we will use holds airport arrivals and departures for airports in the United States for the year 2009. This is a large data set on its own, and will be used for subsequent notebooks in this lesson, as well as future lessons in this course.

The following Code cell first defines the local file name for the data of interest. Next, the second Code cell checks to see if the file exists, and if not uses `wget` to download the data of interest from the appropriate website and stores it in a new file within the appropriate directory.

-----

In [1]:
# Airport 2009 data from stat-computing.org

# First we find our HOME directory
home_dir = !echo $HOME

# Define data directory
data_dir = home_dir[0] +'/data/'

# Second we construct the full path, below our 
# HOME directory to file location
data_file=data_dir + 'airports.csv'

In [2]:
%%bash -s "$data_file"

# Note, we passed in a Python variable above to the Bash script 
# which is then accessed via positional parameter, or $1 in this case.

# First test if file of interest does not exist
if [ ! -f "$1" ] ; then

# If it does not exist, we grab the file from the Internet and
# store it locally in the data directory

wget -O "$1" http://stat-computing.org/dataexpo/2009/airports.csv

else
    
    echo "File already exists locally."
fi

--2017-10-14 19:33:35--  http://stat-computing.org/dataexpo/2009/airports.csv
Resolving stat-computing.org (stat-computing.org)... 54.231.184.247
Connecting to stat-computing.org (stat-computing.org)|54.231.184.247|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244438 (239K) [text/csv]
Saving to: ‘/home/data_scientist/data/airports.csv’

     0K .......... .......... .......... .......... .......... 20%  226K 1s
    50K .......... .......... .......... .......... .......... 41%  333K 1s
   100K .......... .......... .......... .......... .......... 62%  627K 0s
   150K .......... .......... .......... .......... .......... 83%  641K 0s
   200K .......... .......... .......... ........             100%  597K=0.6s

2017-10-14 19:33:36 (402 KB/s) - ‘/home/data_scientist/data/airports.csv’ saved [244438/244438]



-----

Now that the data have been downloaded to our local file system, we can begin to read and write these data by using standard Python tools. The following Code cell uses functions from the `csv` module to read data from a CSV file format into a Python list. We process each row from the file individually, extracting all data form the row and appending the new row to a Python list (`airports`) that contains the data. This process is simplified by using the `with` context to automatically handle opening and closing the file, and by the `reader` function in the `csv` module to iteratively read a single line from the file, which is returned as a list of columns, already tokenized by the specified delimiter, which we specify as a comma via the `delimiter` parameter.

The last line of the Code cell displays three rows from this list (the first three in this case), demonstrating that the `airports` list contains lists of strings, where each sub-list is an individual row, and the strings are the columns. The first row contains the header information.

-----

In [3]:
import csv

airports = []

# Open file and extract flights as list of strings
with open(data_file, 'r') as csvfile:
    
    for row in csv.reader(csvfile, delimiter=','):
        airports.append(row)

# Display first three rows
print(airports[0:3])

[['iata', 'airport', 'city', 'state', 'country', 'lat', 'long'], ['00M', 'Thigpen ', 'Bay Springs', 'MS', 'USA', '31.95376472', '-89.23450472'], ['00R', 'Livingston Municipal', 'Livingston', 'TX', 'USA', '30.68586111', '-95.01792778']]


-----
[[Back to TOC]](#Table-of-Contents)

## Fixed-Width Format Text

The first data format we will discuss are formatted text files. These types of files are seemingly ubiquitous, despite (or perhaps because of) relying on a predominantly ASCII character encoding. This type of data format actually comes in two varieties: fixed-width format, and delimiter separated value format.

Fixed-width format files are preferred by some programming languages, for example, Fortran, and also are easy to view when displayed. But they can be non-trivial to decode in C-based programming languages, including Python. To demonstrate a fixed width format, we can write and subsequently read the airport data we downloaded earlier from the _stat-computing_ website. To write this data as a fixed width data format, we first need to construct appropriate [string formatting codes][sfc].  

In the following code cell, we construct two format code strings, the first one is for the header row that contains the column labels, while the second one is for the data rows. These format codes are fairly easy to understand, we first enclose each string substitution in curly braces `{ }`, and use numbers to indicate the order of substitution. Next, we provide a colon `:` character to indicate the presence of a formatting code, which consists of numbers and a letter code. The numbers following the colon indicate the field width (in characters) and for floating-point data, the numbers after the period specify the precision (or numbers after the decimal point) of the value. The character code indicates the type of data to encode: 's' for string, and 'f' for floating-point. Thus, for example, `{1:29s}` means first format code substitution, with a representation that is 29 characters wide that will accept a string.

Note, we could also use Python's [f-string][pfs] capability, which uses the same format strings to generate a fixed-width format file. In this case, we would simply indicate the variable directly within the curly braces. Thus, rather than `{0:5s}` we have `{row[0]:5s}`. While the f-string approach removes ambiguity and is generally easier to read, the following Code cell uses the first approach, since it results in smaller code for this particular example.

-----

[sfc]: https://docs.python.org/3/library/string.html#format-string-syntax
[pfs]: https://www.python.org/dev/peps/pep-0498/

In [4]:
# First we define our format specification codes
hfmt = "{0:5s}{1:29s}{2:27s}{3:6s}{4:10s}{5:12s}{6:10s}\n"
fmt = "{0:5s}{1:29s}{2:30s}{3:3s}{4:4s}{5:14.8f}{6:14.8f}\n"

# We need to treat the first row special since it is the header row
flag = True

# Now open file and write out airports.
with open(data_dir + 'fixed-width.txt', 'w') as fout:
    for row in airports:

        # We output first line special since it is a header row.
        if flag:
            fout.write(hfmt.format(row[0], row[1], row[2], row[3], row[4], \
                                   row[5], row[6]))
            flag = False
        
        else:
            fout.write(fmt.format(row[0], row[1], row[2], row[3], row[4], \
                                  float(row[5]), float(row[6])))

-----

After running the previous code cell, we have a new file called `fixed-width.txt` that contains the list of airports in a fixed width format file. We can first verify this by either using the Unix command line tools to display the files contents, or simply use the `head` command to see the first few lines.

-----

In [5]:
# Display first few lines from the file
!head -5 $data_dir/fixed-width.txt

iata airport                      city                       state country   lat         long      
00M  Thigpen                      Bay Springs                   MS USA    31.95376472  -89.23450472
00R  Livingston Municipal         Livingston                    TX USA    30.68586111  -95.01792778
00V  Meadow Lake                  Colorado Springs              CO USA    38.94574889 -104.56989330
01G  Perry-Warsaw                 Perry                         NY USA    42.74134667  -78.05208056


-----

One of the hardest parts of dealing with fixed-width format data is quantifying the actual widths of each column. As demonstrated by the previous example, if you know the field widths ahead of time the challenge is greatly simplified. However, for unknown data, simply viewing the data is often not particularly helpful. In this case a good approach is to print out the numerical column values above a few rows of data to simplify determining the widths of each field. This is demonstrated in the following code cell. The code output might wrap around, if so, make your browser wider, until the first row starts with ten spaces, and the second row aligns with the first.

----

In [6]:
# Now lets quantify fixed width fields
# Note the following Code output should start in this manner

#           11111111112222222222
# 012345678901234567890123456789

# We print out the character markers

# First, we create a string of ten space characters, and join to that 
# strings of the numbers 1 - 9, each repeated ten times.

print ((' ' * 10) + ''.join([str(i)*10 for i in range(1, 10)]))

# The second line is simply a repeat of the digits 0-9 ten times, 
# followed by a line of dashes
print('0123456789'*10)
print('-'*100)

# We only want to output a few lines, so we count the rows
rCount = 0

# Now Read in file data.
with open(data_dir + 'fixed-width.txt', 'r') as fin:
    
    for row in fin:
        
        # We skip last character, which is newline
        print(row[:-1])
        
        # We only want to print out first five rows.
        rCount += 1
        if rCount > 5: 
            break

          111111111122222222223333333333444444444455555555556666666666777777777788888888889999999999
0123456789012345678901234567890123456789012345678901234567890123456789012345678901234567890123456789
----------------------------------------------------------------------------------------------------
iata airport                      city                       state country   lat         long      
00M  Thigpen                      Bay Springs                   MS USA    31.95376472  -89.23450472
00R  Livingston Municipal         Livingston                    TX USA    30.68586111  -95.01792778
00V  Meadow Lake                  Colorado Springs              CO USA    38.94574889 -104.56989330
01G  Perry-Warsaw                 Perry                         NY USA    42.74134667  -78.05208056
01J  Hilliard Airpark             Hilliard                      FL USA    30.68801250  -81.90594389


-----

From this output, we can see the widths of each column. For example, the first column extends from column 0 to column 4, while the second column extends across columns 5-33, inclusive. With this information, we could next read each row as a string, pull out the respective characters for each field, strip any leading or trailing white space, and convert the character data to the appropriate format. This approach, however, is left as an exercise for the student (see below). An alternative technique would be to use _Regular Expressions_, discussed in a future lesson to exactly match the requisite number of characters and convert the results appropriately.

-----

[[Back to TOC]](#Table-of-Contents)

## Delimiter Separated Values

A more popular formatted text data format is the delimiter separated value format where a specific delimiter character is used to distinguish between values in different columns. Traditionally, the delimiter most frequently used is the comma, leading to the comma-separated value (CSV) format. However, other delimiters are also used, including whitespace characters like the space or tab characters, or specific, infrequently used characters like the vertical bar `|`. 

We can easily read and write delimiter separated value formats by using the read and rite methods in the [`csv`][csv] module, which was first shown at the start of this notebook. These methods include an optional `delimiter` parameter that can be used to specify the actual value to use to distinguish between consecutive values in row. Other parameters can also be used to control how to escape the delimiter character and how to indicate the end of a line.

-----
[csv]: https://docs.python.org/3/library/csv.html

In [7]:
# We will write a CSV file using the | character as a delimiter
import csv

# Open file and write data as a CSV with the pipe character
with open(data_dir + 'data.csv', 'w') as csvfile:
    
    # We need out csv writer stream
    fout = csv.writer(csvfile, delimiter='|')
    
    # Now write each airport out using the delimiter
    for airport in airports:
        fout.writerow(airport)

-----

This simple code block demonstrated how to write out a vertical-bar
separated value file. We can either view the file contents by using
Unix command line tools or by using the IPython Notebook, as
demonstrated in the next cell.

-----

In [8]:
!head -5 $data_dir/data.csv

----- 

Reading the data into a Python program is straightforward, simply use
the `csv.reader` method to iterate through the rows in the file. We
demonstrate this in the following code cell, where we convert the data
to fixed-width format to improve the readability of the resulting output.

-----

In [9]:
# We can read the data and display by usiung our previous string format codes.
hfmt = "{0:5s}{1:29s}{2:27s}{3:6s}{4:10s}{5:12s}{6:10s}"
fmt = "{0:5s}{1:29s}{2:30s}{3:3s}{4:4s}{5:14.8f}{6:14.8f}"

# First line is header row
rCount = 0

# Now Read in file data.            
with open(data_dir + 'data.csv', 'r') as csvfile:
    for row in csv.reader(csvfile, delimiter='|'):

        # We output first line special since it is a header row.
        if rCount == 0:
            print(hfmt.format(row[0], row[1], row[2], row[3], row[4], \
                              row[5], row[6]))
            print(99*'-')
    
        # Else we simply print the row
        else:
            print(fmt.format(row[0], row[1], row[2], row[3], row[4], \
                             float(row[5]), float(row[6])))
        
        # We only want to print out first five rows.
        rCount += 1
        if rCount > 5: 
            break

iata airport                      city                       state country   lat         long      
---------------------------------------------------------------------------------------------------
00M  Thigpen                      Bay Springs                   MS USA    31.95376472  -89.23450472
00R  Livingston Municipal         Livingston                    TX USA    30.68586111  -95.01792778
00V  Meadow Lake                  Colorado Springs              CO USA    38.94574889 -104.56989330
01G  Perry-Warsaw                 Perry                         NY USA    42.74134667  -78.05208056
01J  Hilliard Airpark             Hilliard                      FL USA    30.68801250  -81.90594389


-----

<font color='red' size = '5'> Student Exercise </font>

Earlier in this notebook, we used the `csv` module to read and write CSV format files and also used format strings to read and write fixed-width format files. Now that you have run the cells in this notebook, go back to the relevant cells and make these changes. Be sure to understand how your changes impact the file input and output process.

3. Try using the hash `#` character as a delimiter.
4. Try using multiple characters, for example the `|` and the `#` as a delimiter. 
2. Change the fixed-width format example to use Python f-strings.
56. The examples reading files treated the data as strings. Change the code to strip leading and trailing white space and to convert all numerical data to floating-point values in the generated list.

-----

## Ancillary Information

The following links are to additional documentation that you might find helpful in learning this material. Reading these web-accessible documents is completely optional.

4. A nice introduction on working with [CSV][1] files from Python
1. An overview of [csv module][2] from Python module of the week
2. Official Python `csv` module [documentation][3] 
43. Wikipedia article on [delimiter separated value files][4] 
32. The [W3C Working group](http://www.w3.org/2013/csvw/wiki/Main_Page) document repository for CSV.

-----

[4]: https://en.wikipedia.org/wiki/Delimiter-separated_values

[1]: http://www.pythonforbeginners.com/systems-programming/using-the-csv-module-in-python/
[2]: https://pymotw.com/2/csv/
[3]: https://docs.python.org/3.6/library/csv.html



**&copy; 2017: Robert J. Brunner at the University of Illinois.**

This notebook is released under the [Creative Commons license CC BY-NC-SA 4.0][ll]. Any reproduction, adaptation, distribution, dissemination or making available of this notebook for commercial use is not allowed unless authorized in writing by the copyright holder.

[ll]: https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode